# Zadanie rekrutacyjne 3 - Regresja logistyczna

Twoim zadaniem jest opracowanie __modelu regresji logistycznej__.

## Analiza danych

Przeanalizuj dane dostarczone w pliku `dane_zadanie_v1.0.xlsx`. 

Czy na ich podstawie zdecydujesz się opracować model regresji logistycznej? Uzasadnij odpowiedź.

Jeśli decydujesz się opracować model napisz także:
- Czy mimo wszystko masz jakieś wątpliwości? 
- Czy brakuje Ci informacji, które mogłyby w Twojej ocenie pomóc?

In [6]:
import pandas as pd

file_path = 'dane_zadanie_v1.0.xlsx'
df = pd.read_excel(file_path)

print(df.head())

print(df.info())
#print(df.describe())

   Zm1       Zm2     Zm3_a     Zm3_b        Zm3_c         Zm3    Zm4  \
0   47  2.556346  0.245325  0.837421  1008.068531  207.098028    NaN   
1   52  5.440271  0.466611  0.184779  1000.823017   86.290555  Kat_d   
2   55  1.907546  0.970312  0.396966  1004.789753  387.025911  Kat_a   
3   51  7.990015  0.104896  0.875934  1004.476626   92.293651  Kat_d   
4   33  1.175730  0.860810  0.458833  1000.042185  394.984532  Kat_a   

         Zm5  Target  
0  14.187720       0  
1  30.193503       1  
2  10.586882       1  
3  44.344586       1  
4   6.525302       1  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Zm1     100 non-null    int64  
 1   Zm2     100 non-null    float64
 2   Zm3_a   100 non-null    float64
 3   Zm3_b   100 non-null    float64
 4   Zm3_c   100 non-null    float64
 5   Zm3     100 non-null    float64
 6   Zm4     77 non-null     o

In [4]:
print(df['Target'].value_counts())


Target
0    64
1    36
Name: count, dtype: int64


Niezrównowazony Target tj. klasa 0 =64 rekordy, klasa 1 = 36.

In [5]:
print(df.isnull().sum())


Zm1        0
Zm2        0
Zm3_a      0
Zm3_b      0
Zm3_c      0
Zm3        0
Zm4       23
Zm5        0
Target     0
dtype: int64


Brakuje 23 wartości Zm4.

Zmienna docelowa (Target) - binarna, ale niezrównowazona, co moze mieć wpływ na ocenę skuteczności modelu
Zmienne predykcyjne (Zm1-Zm5) - wystarczająco danych, by określić zmienność Target
Brakujące dane (23) - przy zastosowaniu technik dla przetwarzania brakującyh danych mozna zminimalizować ten problem (input najczęściej powtarzane/usunąć niepełne rekordy/zmienna dummy)

Na podstawie dostarczonych danych decyduję się na opracowanie modelu regresji logistycznej, mimo problemów związanych z brakującymi danymi, niezrównoważonym rozkładem klas oraz brakiem opisu, co reprezentują poszczególne zmienne (pomogłoby to lepiej zrozumieć kontekst danych), poniewa dane te można skutecznie zaadresować, co pozwala na pełne wykorzystanie potencjału dostępnego inputu.

## Opracowanie modelu

Jeżeli nie zdecydujesz się opracować modelu, pozostaw tę sekcję pustą.

Jeśli decydujesz się opracować model, zamodeluj  `Target` w oparciu o zmienne `Zm1`-`Zm5` (zauważ, że `Zm3_a/b/c` są komponentami zmiennej `Zm3`). 

Możesz dokonywać standardowych przekształceń zmiennych `Zm` aby poprawić jakość działania modelu. 

Przedstaw i krótko opisz kolejne kroki modelowania.

Oceń jakość otrzymanego modelu.

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

numeric_features = ['Zm1', 'Zm2', 'Zm3_a', 'Zm3_b', 'Zm3_c', 'Zm5']
categorical_features = ['Zm4']

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Target']), df['Target'], test_size=0.2, random_state=42)

numeric_transformer = StandardScaler()
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), #uzupełnienie braków przez input='most frequent'
    ('onehot', OneHotEncoder(drop='first'))
])

#model pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', LogisticRegression())])


model.fit(X_train, y_train)

#predykcja i ocena jakości modelu
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[13  0]
 [ 1  6]]
              precision    recall  f1-score   support

           0       0.93      1.00      0.96        13
           1       1.00      0.86      0.92         7

    accuracy                           0.95        20
   macro avg       0.96      0.93      0.94        20
weighted avg       0.95      0.95      0.95        20



Model dokonał 13 poprawnych predykcji dla klasy 0 i 6 poprawnych predykcji dla klasy 1. Istnieje jedno fałszywie pozytywne przewidywanie.
Całkowita dokładność modelu (accuracy) wynosząca 95% oznacza, że model poprawnie sklasyfikował 95% obserwacji w zbiorze testowym.
Wyniki te sugerują, że model dobrze radzi sobie z tym problemem klasyfikacji.